In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import tensorflow as tf
import cv2
import os
import glob
import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Activation,GlobalAveragePooling2D,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow
tf.random.set_seed(100)

In [ ]:
train_path = "../input/covid-data/train/train"
test_path = "../input/covid-data/test/test-2"
SIZE =224

In [ ]:
train_images = []
train_labels =[]
for i in glob.glob( f"{train_path}/*"):
    label = i.split("\\")[-1]
    for j in glob.glob(os.path.join(i,"*.png")):
        img = cv2.imread(j,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)
        
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
test_images = []
test_labels =[]
for i in glob.glob( f"{test_path}/*"):
    label = i.split("\\")[-1]
    for j  in glob.glob(os.path.join(i,"*.png")):
        img = cv2.imread(j,cv2.IMREAD_COLOR)
        img = cv2.resize(img,(SIZE,SIZE))
        img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(label)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
# from tensorflow.keras.applications.resnet50 import ResNet50
# conv_base = ResNet50(weights='imagenet',include_top =False,
#                   input_shape=(SIZE,SIZE,3))

In [ ]:
# for layer in conv_base.layers:
#   layer.trainable =False
# conv_base.summary()

In [ ]:
# feature_map_model = Model(inputs=conv_base.input, outputs=conv_base.get_layer('conv5_block3_out').output)

In [ ]:
train_features = np.load('../input/covid-resnet50-embeds/train_feats.npy')
test_features = np.load('../input/covid-resnet50-embeds/test_feats.npy')

In [ ]:
# train_features = feature_map_model.predict(train_images)
# test_features = feature_map_model.predict(test_images)
print(np.max(train_features))
print(train_features.mean())
print(np.max(test_features))
print(test_features.mean())
print(train_features.std())
print(test_features.std())

In [ ]:
train_features.shape

In [ ]:
# np.save('train_feats.npy',train_features)

In [ ]:
# np.save('test_feats.npy',test_features)

In [ ]:
# print(train_features.resize(3201,100352))
# print(test_features.resize(1147,100352))
# # from sklearn.preprocessing import RobustScaler
# # scaler = RobustScaler()
# # # # transform data
# # train_features_scaled = scaler.fit_transform(train_features)
# # test_features_scaled = scaler.transform(test_features)

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# # # # transform data
# train_features_scaled = scaler.fit_transform(train_features)
# test_features_scaled = scaler.transform(test_features)



# print(train_features_scaled.mean())
# print(test_features_scaled.mean())




In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
weights = compute_class_weight('balanced', np.unique(train_labels_encoded),train_labels_encoded)
cw = dict(zip( np.unique(train_labels_encoded), weights))
print(cw)

In [ ]:
train_features.shape

In [ ]:
test_features2 = test_features.reshape(-1,49,2048)
train_features2 = train_features.reshape(-1,49,2048)

---------------**RANDOM** **FOREST**---------------------------

--------------------------------------**CUSTOMISED** **ANN**---------------------------------------------

In [ ]:
# model = Sequential()
# model.add(Flatten(input_shape = (7,7,2048)))
# model.add(Dropout(0.3))
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.3))
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.3))

# model.add(Dense(1, activation='sigmoid'))
# model.summary()



In [ ]:
from keras.layers import Dense, Flatten, Convolution1D, Dropout


In [ ]:
model = Sequential()
# model.add(Flatten(input_shape = (7,7,2048)))
model.add(keras.layers.Input(shape=train_features2.shape[-2:]))
model.add(keras.layers.Conv1D(filters=1024, kernel_size=4, activation='relu'))
model.add(keras.layers.BatchNormalization())


model.add(Flatten())


model.add(Dense(2048, activation='relu'))
model.add(keras.layers.BatchNormalization())


# model.add(Dropout(0.3))
model.add(Dropout(0.3))


model.add(Dense(1024, activation='relu'))
model.add(keras.layers.BatchNormalization())


# model.add(Dropout(0.3))
model.add(Dropout(0.3))


model.add(Dense(512, activation='relu'))
model.add(keras.layers.BatchNormalization())


model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(keras.layers.BatchNormalization())


model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(keras.layers.BatchNormalization())


model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(keras.layers.BatchNormalization())


model.add(Dropout(0.3))

model.add(Dense(1, activation='sigmoid'))
model.summary()


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
early = EarlyStopping(monitor="val_loss", mode= "min", patience=7)
learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss", patience =2, verbose=1,factor=0.2, min_learning_rate=0.000001)
callbacks_list = [ early,learning_rate_reduction]
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
  

# history = model.fit(train_features,train_labels_encoded, batch_size =100 , callbacks= callbacks_list , class_weight = cw , epochs=100, validation_data= (test_features,test_labels_encoded))
history = model.fit(train_features2,train_labels_encoded, batch_size =100 , callbacks= callbacks_list , class_weight = cw , epochs=100, validation_data= (test_features2,test_labels_encoded))



In [ ]:
# pd.DataFrame(model.history.history).plot()

In [ ]:
np.mean(train_features)

In [ ]:
np.max(train_features)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
preds  = model.predict(test_features2)
# roc curve for models




In [ ]:
# import decimal
# from sklearn.metrics import roc_curve, roc_auc_score
# from sklearn import metrics
# def float_range(start, stop , step):
#   while start < stop:
#     yield float(start)
#     start += decimal.Decimal(step)





# for x in list(float_range(0, 1, '0.01')):

#   predictions = preds.copy()
#   predictions[predictions <= x] = 0
#   predictions[predictions > x] = 1
#   print(roc_auc_score(test_labels_encoded, predictions),x) 

fpr1, tpr1, thresh1 = roc_curve(test_labels_encoded,preds)

In [ ]:
predictions = preds.copy()
predictions[predictions <= 0.13] = 0
predictions[predictions > 0.13] = 1


In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_true=test_labels_encoded , y_pred=predictions,target_names =['Covid','Normal']))



In [ ]:
random_probs = [0 for i in range(len(test_labels_encoded))]
p_fpr, p_tpr, _ = roc_curve(test_labels_encoded, random_probs, pos_label=1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
cm = pd.DataFrame(data=confusion_matrix(test_labels_encoded, predictions, labels=[0, 1]),index=["Actual Covid", "Actual Normal"],
columns=["Predicted Covid", "Predicted Normal"])
import seaborn as sns
sns.heatmap(cm,annot=True,fmt="d")

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
t = f.suptitle('Resnet50 transfer_learning Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

ax1.plot( history.history['accuracy'], label='Train Accuracy')
ax1.plot( history.history['val_accuracy'], label='Validation Accuracy')

ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot( history.history['loss'], label='Train Loss')
ax2.plot( history.history['val_loss'], label='Validation Loss')
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")

In [ ]:
 #plot roc curves
plt.plot(fpr1, tpr1, marker='.',color='red',label = "Resnet50 + ANN")
# plt.plot(fpr2, tpr2, marker='.',color='BLACK',label = "Resnet50 + RANDOM_FOREST")
# plt.plot(fpr3, tpr3, marker='.',color='violet',label = "Resnet50 + Logistic_regression", linewidth =3)
# plt.plot(fpr4, tpr4, marker='.',color='green',label = "Resnet50 + KNN ")
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue',label = "No_skill")

# title
plt.title('ROC curves ')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();


In [ ]:
auc = roc_auc_score(test_labels_encoded, preds)
print('AUC of "RESNET50 + ANN_1": %.4f' % auc)

In [ ]:
train_images/255